# Demo Peg Keeping Logic

One of the agents in our simulation is the `Keeper`. The `Keeper` calls the `update` function on all Peg Keepers if doing so exceeds their profitability threshold (default 0). This notebook demos the `Keeper`'s `update` method.

In [1]:
from src.sim.scenario import Scenario

In [2]:
def increment_timestamps(aggregator, pks):
    ts = aggregator._block_timestamp + 60 * 60  # one hour
    for pk in pks:
        pk._increment_timestamp(ts)
        pk.POOL._increment_timestamp(ts)
        assert pk.last_change != pk._block_timestamp
    aggregator._increment_timestamp(ts)
    print(f"New Aggregator Price: {aggregator.price() / 1e18}")

In [3]:
# Generate markets and prices
scenario = Scenario("baseline", "wsteth")
scenario.prepare_for_run()

[INFO][19:09:32][src.sim.scenario]-17281: Fetching sim_market from subgraph.
[INFO][19:09:41][src.utils.poolgraph]-17281: Found 20 valid trading cycles of length 3.


In [4]:
aggregator = scenario.aggregator
aggregator.price() / 1e18

0.9999613106149258

In [5]:
pks = scenario.peg_keepers
keeper = scenario.keeper
profit, count = keeper.update(pks)
print(f"Profit: {profit}, Count: {count}")

Profit: 0.0, Count: 0


In all likelihood, no updates occured even though the pools are not balanced. This is because the balance of crvUSD exceeds the balance of the peg stablecoin (e.g. USDC), but the `PegKeeper` doesn't have any debt to withdraw. To test the `update` functionality, we trade against the stableswap pools such that the crvUSD balance is less than the peg stablecoin balance.

In [6]:
for pk in pks:
    spool = pk.POOL
    print(spool.name)

    normalized_balances = [b * r / 1e18 for b, r in zip(spool.balances, spool.rates)]
    print(f"Normalized balances before trade: {normalized_balances}")

    diff = normalized_balances[pk.I] - normalized_balances[pk.I ^ 1]
    if diff < 1:
        continue

    diff = int(diff * 1e18 / spool.rates[pk.I ^ 1])  # convert to peg coin units
    print(
        f"Swapping in {diff} {spool.coins[pk.I^1].symbol} for {spool.coins[pk.I].symbol}"
    )

    amt_in, amt_out, fees = spool.trade(pk.I ^ 1, pk.I, diff)
    assert amt_in == diff
    print(f"Received: {amt_out}, Fees: {fees}")
    normalized_balances = [b * r / 1e18 for b, r in zip(spool.balances, spool.rates)]
    print(f"Normalized balances after trade: {normalized_balances}")

    print()

Curve.fi Factory Plain Pool: crvUSD/TUSD
Normalized balances before trade: [1.717754483507028e+24, 2.762115253867853e+24]
Swapping in 1044360770360825199198208 TUSD for crvUSD
Received: 1044256334283789116299323, Fees: 104436077036082519881
Normalized balances after trade: [2.762115253867853e+24, 1.7178067015455457e+24]

Curve.fi Factory Plain Pool: crvUSD/USDP
Normalized balances before trade: [1.842022411531397e+24, 2.3000336919881115e+24]
Swapping in 458011280456714396106752 USDP for crvUSD
Received: 457965479328668724605693, Fees: 45801128045671439604
Normalized balances after trade: [2.3000336919881118e+24, 1.8420453120954198e+24]

Curve.fi Factory Plain Pool: crvUSD/USDC
Normalized balances before trade: [1.7626578665113e+25, 1.9319573233956232e+25]
Swapping in 1692994568843 USDC for crvUSD
Received: 1692825269386115742757538, Fees: 169299456884300004276
Normalized balances after trade: [1.9319573233956e+25, 1.7626663314841675e+25]

Curve.fi Factory Plain Pool: crvUSD/USDT
Normal

In [7]:
# Increment timestamps to update aggregator price
# AND pk timestamps
increment_timestamps(aggregator, pks)

New Aggregator Price: 1.0002764113935887


In [8]:
profit, count = keeper.update(pks)
print(f"Profit: {profit / 1e18}, Count: {count}")

Profit: 4.608109929655284, Count: 2


In [9]:
for pk in pks:
    spool = pk.POOL
    print(spool.name)

    normalized_balances = [b * r / 1e18 for b, r in zip(spool.balances, spool.rates)]
    print(f"Normalized balances before trade: {normalized_balances}")
    print()

Curve.fi Factory Plain Pool: crvUSD/TUSD
Normalized balances before trade: [2.762115253867853e+24, 1.7178067015455457e+24]

Curve.fi Factory Plain Pool: crvUSD/USDP
Normalized balances before trade: [2.300032420134736e+24, 1.9336417167308543e+24]

Curve.fi Factory Plain Pool: crvUSD/USDC
Normalized balances before trade: [1.9319568807374e+25, 1.796524087281051e+25]

Curve.fi Factory Plain Pool: crvUSD/USDT
Normalized balances before trade: [1.5021145364448999e+25, 1.2873925066796794e+25]



In [10]:
# Increment timestamps to update aggregator price
# AND pk timestamps
increment_timestamps(aggregator, pks)

New Aggregator Price: 1.0002538788444169


In [11]:
# We can update again! Recall that the PK can only
# deposit/withdraw 20% of the pool's imbalance at a time.
profit, count = keeper.update(pks)
print(f"Profit: {profit / 1e18}, Count: {count}")

Profit: 2.399235690071639, Count: 2


In [12]:
# Increment timestamps to update aggregator price
# AND pk timestamps
increment_timestamps(aggregator, pks)

New Aggregator Price: 1.000233227377818
